In [56]:
def factorizacion_berlekamp_char2(f, max_intentos=5, verbose=False):
    """
    Versión adaptada del algoritmo de Berlekamp para q = 2^k (característica 2).

    Parámetros:
        f (PolynomialElement): polinomio en GF(2^k)[x], libre de cuadrados.
        max_intentos (int)   : número máximo de intentos aleatorios para separar un factor.

    Retorna:
        List[PolynomialElement]: lista de factores irreducibles (mónicos).
    """

    # 0) Comprobaciones preliminares
    R = f.parent()          
    K = R.base_ring()       
    if not K.is_finite() or K.characteristic() != 2:
        raise ValueError("Este algoritmo está adaptado sólo para GF(2^k).")
    
    f = f.monic()
    if f.degree() <= 1:
        return [f]

    # Comprobar libre de cuadrados
    # if gcd(f, f.derivative()) != 1:
    #    raise ValueError("f no está libre de cuadrados. Se requiere factor libre de cuadrados.")
    
    # Función para elegir un α "aleatorio" en R = GF(2^k)[x]/(f):
    # Para simplicidad, generamos un polinomio con coeficientes aleatorios en GF(2^k),
    # y lo reducimos mod f.
    def alfa_aleatorio_en_R():
        degf = f.degree()
        pol = R.random_element(degf-1)
        return pol % f  # la clase de pol mod f
    
    # Función gcd de "separación": dada un factor poly_f y un α, tratamos de
    # encontrar un gcd no trivial para factorizar poly_f.
    def separar_con_alfa(poly_f, alfa):
        """
        Intenta separar poly_f usando un gcd con (alfa^2 - alfa), (alfa^4 - alfa), etc.
        (Es una simplificación didáctica inspirada en la idea T_k(α).)
        Retorna [poly_f] si no se separa, o [g, poly_f//g] si se separa.
        """
        # Ejemplo: comprobamos varias potencias "Frobenius" de α
        # (en caracter 2, la potencia p = 2).
        for i_potencia in range(1, poly_f.degree()+1):
            h = (alfa^(2^i_potencia) - alfa) % poly_f
            g = gcd(poly_f, h)
            if g.degree() not in (0, poly_f.degree()):
                return [g.monic(), (poly_f//g).monic()]
        return [poly_f]
    
    # Bucle principal: Recursividad
    pendientes = [f]
    factores_irreducibles = []
    while pendientes:
        actual = pendientes.pop()
        actual = actual.monic()
        
        if verbose:
            print("============================")
            print(f"Procesando polinomio: {actual}")
            print("============================")
        
        if actual.degree() <= 1:
            if verbose:
                print(f"El polinomio {actual} es irreducible.")
            factores_irreducibles.append(actual)
            continue
        
        # Intentamos separarlo escogiendo hasta max_intentos elementos α "aleatorios"
        separados = [actual]
        separacion_encontrada = False
        for intento in range(max_intentos):
            alfa = alfa_aleatorio_en_R()
            
            if verbose:
                print(f"({intento}). Probando con α = {alfa}")
                print("----------------------------")
            
            nueva_separacion = []
            for trozo in separados:
                if trozo.degree() <= 1 or trozo.is_irreducible():
                    nueva_separacion.append(trozo)
                else:
                    partes_separadas = separar_con_alfa(trozo, alfa)
                    nueva_separacion.extend(partes_separadas)
            separados = nueva_separacion
            
            if verbose:
                separados.sort(key=lambda x: x.degree())
                print(f"Divisiones obtenidas: {separados}")
                print("----------------------------")
            
            # Si en algún punto efectivamente se dividió el polinomio -> break
            if len(separados) > 1:
                separacion_encontrada = True
                break
        
        # Si no lo pudimos separar, asumimos que ya es irreducible 
        # (o no se logró separar con max_intentos. Probablemente queramos aumentar max_intentos).
        if not separacion_encontrada and len(separados) == 1:
            if verbose:
                print(f"No se pudo dividir {actual}. Asumiendo irreducible.")
                print("============================")
            factores_irreducibles.append(actual)
        else:
            if verbose:
                separados.sort(key=lambda x: x.degree())
                print(f"Polinomios resultantes a procesar: {separados}")
                print("============================")
            # puede haber varios trozos -> re-enviarlos a la cola
            pendientes.extend(separados)
    
    return factores_irreducibles


EL QEU IBA A ENTREGAR

In [ ]:
def factorizacion_berlekamp_char2(f, max_intentos=20, verbose=False):
    """
    Versión adaptada del algoritmo de Berlekamp para q = 2^k (característica 2).

    Parámetros:
        f (PolynomialElement): polinomio en GF(2^k)[x], libre de cuadrados.
        max_intentos (int)   : número máximo de intentos aleatorios para separar un factor.
        verbose (bool)       : Indicador para mostrar proceso del algoritmo

    Retorna:
        List[PolynomialElement]: lista de factores irreducibles (mónicos).
    """

    # 0) Comprobaciones preliminares
    R = f.parent()          
    K = R.base_ring()       
    if not K.is_finite() or K.characteristic() != 2:
        raise ValueError("Este algoritmo está adaptado sólo para GF(2^k).")
    
    # Forzamos f mónico
    f = f.monic()
    # Comprobar libre de cuadrados (no es necesario po eso lo he comentado, en un pricipio lo puse por hipótesis)
    # if gcd(f, f.derivative()) != 1:
    #    raise ValueError("f no es libre de cuadrados")

    # Caso trivial
    if f.degree() <= 1:
        return [f]

    # 1) Funciones internas
    # Función para elegir α aleatorio
    def alfa_aleatorio_en_R(poly):
        degf = f.degree()
        pol = R.random_element(degf-1)
        return pol % poly
    
    # Función de "separación": dado un factor poly_f y un α, tratamos de
    # encontrar un gcd no trivial para factorizar poly_f.
    def separar_con_alfa(poly_f, alfa):
        for i_potencia in range(1, poly_f.degree()+1):
            h = (alfa^(2^i_potencia) - alfa) % poly_f
            g = gcd(poly_f, h)
            if g.degree() not in (0, poly_f.degree()):
                return [g.monic(), (poly_f//g).monic()]
        return [poly_f]

    # 2) Bucle principal
    pendientes = [f]
    factores_irreducibles = []
    while pendientes:
        actual = pendientes.pop()
        actual = actual.monic()
        
        if verbose:
            print("============================")
            print(f"Procesando polinomio: {actual}")
            print("============================")
        
        if actual.degree() <= 1:
            if verbose:
                print(f"El polinomio {actual} es irreducible.")
            factores_irreducibles.append(actual)
            continue
        
        # Intentamos separarlo escogiendo hasta max_intentos elementos α
        separados = [actual]
        separacion_encontrada = False
        for intento in range(max_intentos):            
            
            
            nueva_separacion = []
            for trozo in separados:
                if trozo.degree() <= 1 or trozo.is_irreducible():
                    nueva_separacion.append(trozo)
                else:
                    alfa = alfa_aleatorio_en_R(trozo)
                    if verbose:
                        print(f"({intento}). Probando con α = {alfa}")
                        print("----------------------------")
                    partes_separadas = separar_con_alfa(trozo, alfa)
                    nueva_separacion.extend(partes_separadas)
            separados = nueva_separacion
            
            if verbose:
                separados.sort(key=lambda x: x.degree())
                print(f"Divisiones obtenidas: {separados}")
                print("----------------------------")
            
            # Si en algún punto efectivamente se dividió el polinomio -> break
            if len(separados) > 1:
                separacion_encontrada = True
                break
        
        # Si no lo pudimos separar, asumimos que ya es irreducible 
        # (o no se logró separar con max_intentos. Probablemente queramos aumentar max_intentos).
        if not separacion_encontrada and len(separados) == 1:
            if verbose:
                print(f"No se pudo dividir {actual}. Asumiendo irreducible.")
                print("============================")
            factores_irreducibles.append(actual)
        else:
            if verbose:
                separados.sort(key=lambda x: x.degree())
                print(f"Polinomios resultantes a procesar: {separados}")
                print("============================")
            # puede haber varios trozos -> re-enviarlos a la cola
            pendientes.extend(separados)
    
    return factores_irreducibles

In [57]:
K = GF(2)
R.<x> = PolynomialRing(K, "x")
f = x^8+x^6+x^4+x^3+1
f = x^126 + x^124 + x^120 + x^118 + x^114 + x^108 + x^107 + x^105 + x^104 + x^100 + x^98 + x^90 + x^88 + x^86 + x^84 + x^83 + x^82 + x^81 + x^80 + x^79 + x^78 + x^77 + x^75 + x^73 + x^72 + x^71 + x^67 + x^66 + x^65 + x^62 + x^60 + x^57 + x^55 + x^52 + x^50 + x^49 + x^47 + x^46 + x^43 + x^42 + x^39 + x^38 + x^35 + x^34 + x^33 + x^32 + x^31 + x^27 + x^26 + x^25 + x^23 + x^22 + x^20 + x^19 + x^15 + x^13 + x^12 + x^10
factors_berlekamp = factorizacion_berlekamp_char2(f, verbose=True)
resu = list(set(factors_berlekamp))
resu.sort()
print(len(resu))
resu

Procesando polinomio: x^126 + x^124 + x^120 + x^118 + x^114 + x^108 + x^107 + x^105 + x^104 + x^100 + x^98 + x^90 + x^88 + x^86 + x^84 + x^83 + x^82 + x^81 + x^80 + x^79 + x^78 + x^77 + x^75 + x^73 + x^72 + x^71 + x^67 + x^66 + x^65 + x^62 + x^60 + x^57 + x^55 + x^52 + x^50 + x^49 + x^47 + x^46 + x^43 + x^42 + x^39 + x^38 + x^35 + x^34 + x^33 + x^32 + x^31 + x^27 + x^26 + x^25 + x^23 + x^22 + x^20 + x^19 + x^15 + x^13 + x^12 + x^10
(0). Probando con α = x^125 + x^124 + x^118 + x^114 + x^113 + x^108 + x^103 + x^101 + x^100 + x^93 + x^90 + x^89 + x^88 + x^87 + x^85 + x^84 + x^83 + x^82 + x^81 + x^80 + x^79 + x^78 + x^74 + x^70 + x^66 + x^65 + x^64 + x^62 + x^60 + x^59 + x^58 + x^57 + x^56 + x^54 + x^53 + x^50 + x^48 + x^47 + x^45 + x^44 + x^41 + x^40 + x^37 + x^35 + x^34 + x^33 + x^27 + x^26 + x^22 + x^16 + x^13 + x^12 + x^11 + x^10 + x^9 + x^8 + x^6 + x^5
----------------------------
Divisiones obtenidas: [x^8 + x^7 + x^6 + x^5, x^118 + x^117 + x^116 + x^115 + x^114 + x^113 + x^106 + x^

[x,
 x + 1,
 x^2 + x,
 x^2 + x + 1,
 x^3 + x + 1,
 x^3 + x^2 + 1,
 x^4 + x^3 + 1,
 x^5 + x^3 + x^2 + x + 1,
 x^5 + x^4 + x^2 + x + 1,
 x^5 + x^4 + x^3 + x + 1,
 x^5 + x^4 + x^3 + x^2 + 1,
 x^6 + x^4 + x^3 + x + 1,
 x^6 + x^5 + x^2 + x + 1,
 x^6 + x^5 + x^3 + x^2 + 1]

# Test

In [54]:
# Test 1
K = GF(2)
R.<x> = PolynomialRing(K, "x")
f = x^8+x^6+x^4+x^3+1
factors_berlekamp = factorizacion_berlekamp_char2(f)
factors_berlekamp.sort()
print('Bruto:', factors_berlekamp)
resu = list(set(factors_berlekamp))
resu.sort()
print(len(resu))
resu

Bruto: [x^2 + x + 1, x^6 + x^5 + x^4 + x + 1]
2


[x^2 + x + 1, x^6 + x^5 + x^4 + x + 1]

In [53]:
# Test 2
K = GF(2)
R.<x> = PolynomialRing(K, "x")
f = x^10+x^9+x^3+x^2+1
factors_berlekamp = factorizacion_berlekamp_char2(f)
factors_berlekamp.sort()
print('Bruto:', factors_berlekamp)
resu = list(set(factors_berlekamp))
resu.sort()
print(len(resu))
resu

Bruto: [x^2 + x + 1, x^2 + x + 1, x^2 + x + 1, x^4 + x + 1]
2


[x^2 + x + 1, x^4 + x + 1]

In [55]:
# Test 23
K = GF(2)
R.<x> = PolynomialRing(K, "x")
f = x^4 + x + 1
factors_berlekamp = factorizacion_berlekamp_char2(f)
factors_berlekamp.sort()
print('Bruto:', factors_berlekamp)
resu = list(set(factors_berlekamp))
resu.sort()
print(len(resu))
resu

Bruto: [x^4 + x + 1]
1


[x^4 + x + 1]

In [38]:
print(len(factors_berlekamp))
result = list(set(factors_berlekamp))
result.sort()
print(len(result))
result

48
14


[x,
 x + 1,
 x^2 + x,
 x^2 + x + 1,
 x^3 + x + 1,
 x^3 + x^2 + 1,
 x^4 + x^3 + 1,
 x^5 + x^3 + x^2 + x + 1,
 x^5 + x^4 + x^2 + x + 1,
 x^5 + x^4 + x^3 + x + 1,
 x^5 + x^4 + x^3 + x^2 + 1,
 x^6 + x^4 + x^3 + x + 1,
 x^6 + x^5 + x^2 + x + 1,
 x^6 + x^5 + x^3 + x^2 + 1]

In [41]:
result==resu

True